# Preprocess obesity challenge data
This notebook preprocesses MIMIC-III and obesity data to make them ready for apache cTAKES CUI extraction, and extract CUIs from the output xmi files of cTAKES process.

Steps:
1. Process MIMIC-III files
2. Process Obesity data - convert xml file to txt
2. Use Ctake to get CUIs (xmi file)
3. read xmi file, extract CUIs, and convert to txt 
 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import xml.etree.ElementTree as ET
#https://docs.python.org/3/library/xml.etree.elementtree.html
import pandas as pd
import re
import shutil
import numpy as np
import sys
import os, os.path
import shutil

# Step 1: Preprocess MIMIC-III data for CUI extraction

In [ ]:
notes = pd.read_csv("drive/MyDrive/mimic-iii/NOTEEVENTS.csv")
sorted_notes = notes.sort_values("SUBJECT_ID").reset_index(drop=True)
sorted_notes.head()

In [ ]:
sorted_notes["TEXT"] = sorted_notes["TEXT"].str.replace("\n", " ")
sorted_notes.head()

In [ ]:
for i in range(len(sorted_notes)):
  text = sorted_notes.loc[i, "TEXT"]
  # these illegal chars block the CUI extraction of apache ctakes
  illegal_xml_chars_re = re.compile(u'[\x00-\x08\x0b\x0c\x0e-\x1F\uD800-\uDFFF\uFFFE\uFFFF]')
  if illegal_xml_chars_re.search(text):
    print(i,sorted_notes.loc[i, "SUBJECT_ID"], sorted_notes.loc[i, "TEXT"])
    new_text = illegal_xml_chars_re.sub('', text)
    sorted_notes.loc[i, "TEXT"] = new_text


In [ ]:
#go through sorted table and write each patient's aggregated clinical notes into a file
file_str = sorted_notes.loc[0, "TEXT"]
query_patient = sorted_notes.loc[0, "SUBJECT_ID"]

for i in range(1, len(sorted_notes)):
  if (i % 10000) == 0:
    print(i)

  curr_patient = sorted_notes.loc[i, "SUBJECT_ID"]
  if curr_patient == query_patient:
    file_str += " " + sorted_notes.loc[i, "TEXT"]
  else:
    with open("drive/MyDrive/mimic-iii/notes_2/patient_{}.txt".format(query_patient), "w") as f:
      f.write(file_str)
    query_patient = curr_patient
    file_str = sorted_notes.loc[i, "TEXT"]

#handle end
with open("drive/MyDrive/mimic-iii/notes_2/patient_{}.txt".format(query_patient), "w") as f:
  f.write(file_str)

In [ ]:
# saving processed files
# path joining version for other paths
DIR = 'drive/MyDrive/mimic-iii/notes_2'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

In [ ]:
shutil.make_archive('drive/MyDrive/mimic-iii/notes_patients_2', 'zip', 'drive/MyDrive/mimic-iii/notes_2')

# Step 2: Convert raw Obesity Challenge data to individual txt per patient

In [ ]:
# convert xml to individual txt for ctakes pipeline to process and extract CUI
def xml2txts(file_path, out_path): 
  text = ET.parse(file_path)
  for patient in text.iter('doc'):
  #print(patient.attrib)
    uid = patient.attrib['id']
    notes = patient[0].text
    path = f"{out_path}{uid}.txt"
    with open(path, 'w') as f:
      f.write(notes)

In [ ]:
file_path_test = '/content/drive/My Drive/Obesity challenge/obesity_patient_records_test.xml'
out_path_test = "/content/drive/My Drive/Obesity challenge/test_data_txt/"

In [ ]:
# process test data into files
xml2txts(file_path_test, out_path_test)

In [ ]:
# process train data into txt files
file_path_train1 = '/content/drive/My Drive/Obesity challenge/obesity_patient_records_training.xml'
out_path_train1 = "/content/drive/My Drive/Obesity challenge/train_data_txt/"
file_path_train2 = '/content/drive/My Drive/Obesity challenge/obesity_patient_records_training2.xml'
out_path_train2 = "/content/drive/My Drive/Obesity challenge/train_data2_txt/"

In [ ]:
xml2txts(file_path_train1, out_path_train1)
xml2txts(file_path_train2, out_path_train2)

# Step 3: Process txt files with clinical notes with Apache ctakes (offline)

# Step 4: Extract CUIs from processed xmi files

In [ ]:
def extractCUIs4Patient(patient_xmi_path='/content/drive/My Drive/Obesity challenge/test_data_output/3.txt.xmi'):
  singleDoc = ET.parse(patient_xmi_path)
  patient_id = int(re.findall('\d+(?=.txt.xmi)',patient_xmi_path)[0])
  res = []
  for unit in singleDoc.iter("{http:///org/apache/ctakes/typesystem/type/refsem.ecore}UmlsConcept"):
    current_cui = unit.attrib['cui']
    res.append(current_cui)
  result = " ".join(res)
  return patient_id, result

In [ ]:
mimic_path = "/content/drive/My Drive/MIMIC/output_first_half_selected/"
for file in os.listdir(mimic_path):
  file_path = mimic_path+file
  try:
    id, res = extractCUIs4Patient(file_path)
    with open(f'/content/drive/My Drive/MIMIC/output_first_half_selected_cuis/patient_{id}.txt', 'w') as f:
      f.write(res)
  except:
    print ("An exeption occured")
    continue
  i+=1
  if i%100==0:
    print(f"finished extracting {i} files...")

finished extracting 400 files...
finished extracting 500 files...
An exeption occured
finished extracting 600 files...
finished extracting 700 files...
An exeption occured
An exeption occured
An exeption occured
An exeption occured
finished extracting 800 files...
finished extracting 900 files...
An exeption occured
finished extracting 1000 files...
An exeption occured
finished extracting 1100 files...
An exeption occured
An exeption occured
finished extracting 1200 files...
An exeption occured
finished extracting 1300 files...
finished extracting 1400 files...
An exeption occured
finished extracting 1500 files...
An exeption occured
finished extracting 1600 files...
An exeption occured
An exeption occured
finished extracting 1700 files...
An exeption occured


In [ ]:
#for obesity data
i=0
train_data_path = "/content/drive/My Drive/Obesity challenge/train_data_output2/"
for file in os.listdir(train_data_path):
  file_path = train_data_path+file
  id, res = extractCUIs4Patient(file_path)
  with open(f'/content/drive/My Drive/Obesity challenge/train_data_cuis/{id}.txt', 'w') as f:
    f.write(res)
  i+=1
  if i%10==0:
    print(f"finished extracting {i} files...")

finished extracting 10 files...
finished extracting 20 files...
finished extracting 30 files...
finished extracting 40 files...
finished extracting 50 files...
finished extracting 60 files...
finished extracting 70 files...
finished extracting 80 files...
finished extracting 90 files...
finished extracting 100 files...
finished extracting 110 files...


In [ ]:
#for obesity data test
i=0
test_data_path = "/content/drive/My Drive/Obesity challenge/test_data_output/"
for file in os.listdir(test_data_path):
  file_path = test_data_path+file
  id, res = extractCUIs4Patient(file_path)
  with open(f'/content/drive/My Drive/Obesity challenge/test_data_cuis/{id}.txt', 'w') as f:
    f.write(res)
  i+=1
  if i%10==0:
    print(f"finished extracting {i} files...")

finished extracting 10 files...
finished extracting 20 files...
finished extracting 30 files...
finished extracting 40 files...
finished extracting 50 files...
finished extracting 60 files...
finished extracting 70 files...
finished extracting 80 files...
finished extracting 90 files...
finished extracting 100 files...
finished extracting 110 files...
finished extracting 120 files...
finished extracting 130 files...
finished extracting 140 files...
finished extracting 150 files...
finished extracting 160 files...
finished extracting 170 files...
